In [ ]:
import numpy as np
import pandas as pd

# to read data from a csv file
music = pd.read_csv("SpotifyFeatures3.csv")
# storing only necessary features
music = music[['genre','artist_name','track_name','track_id','popularity']]

# removing all the spaces(Eg: if there is a singer like Alan Walker, Alan and Walker will be treated as two different singers)
music['genre']=music['genre'].apply(lambda x:x.replace(" ",""))
music['artist_name']=music['artist_name'].apply(lambda x:x.replace(" ",""))
music['track_name']=music['track_name'].apply(lambda x:x.replace(" ",""))

# creating tags to find out similarity of one song with all other songs
music['tags'] = music['genre']+ ", "+ music['artist_name']
music_new = music[['genre','artist_name','track_name','track_id','popularity','tags']] 

# stemming 
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)
music_new['tags'] = music_new['tags'].apply(stem)

#extracting the similarity between one song and all other songs and storing them in vector format
from sklearn.feature_extraction.text import *
cv = CountVectorizer(max_features=10000,stop_words='english')
vectors = cv.fit_transform(music_new['tags']).toarray()

# finding cosine distnace between the vectors that is the angle between the vectors
from sklearn.metrics.pairwise import *
cosine_similarity(vectors)
similarity=cosine_similarity(vectors)
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

#function to recommend song based on song
def recommend(song):
  music_index = music_new[music_new['track_name']==song].index[0]
  distances = similarity[music_index]
  music_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
  for i in music_list:
    print(music_new.iloc[i[0]].track_name)

##function to recommend song based on singer
def recommend1(singer):
  music_index = music_new[music_new['artist_name']==singer].index[0]
  distances = similarity[music_index]
  music_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
  for i in music_list:
    print(music_new.iloc[i[0]].track_name)

#function to recommend song based on genre
def recommend2(genre):
  music_index = music_new[music_new['genre']==genre].index[0]
  distances = similarity[music_index]
  music_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])
  for i in music_list:
    print(music_new.iloc[i[0]].track_name)

print("MUSIC RECOMMENDATION SYSTEM")
print("1. Based on Song")
print("2. Based on singer")
print("3. Based on genre")
print()
ch = int(input("Enter Choice: "))
if ch ==1:
  song = input("Enter a song: ")
  print()
  print("Here are a few songs you, happy listening :)")
  recommend(song)
elif ch == 2:
  singer = input("Enter a singer: ")
  print()
  print("Here are a few songs you, happy listening :)")
  recommend1(singer)
elif ch ==3:
  genre = input("Enter a genre: ")
  print()
  print("Here are a few songs you, happy listening :)")
  recommend2(genre)
else:
  print("Invalid choice")